## Bootcamp Homework 6 [pn2330@nyu.edu] 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

In [3]:
data = pd.read_csv("/Users/pranav/Desktop/Datascience_Bootcamp_Spring_2025/Homework_6_pn2330/employee.csv")

In [4]:
data.shape

(4277, 13)

In [5]:
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [6]:
# dropping id, timestamp and country columns
data = data.drop(columns=['id', 'timestamp','country'])

In [7]:
data.columns

Index(['employment_status', 'job_title', 'job_years', 'is_manager',
       'hours_per_week', 'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')

In [8]:
# replacing NANs in hours_per_week with median value of the column
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()

In [9]:
#Handling null values in categorical columns
data = data.dropna()

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


###Data Visualization :

In [12]:
data_train = data.copy()
data_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [13]:
# selecting categorical features
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols

['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [14]:
#Encoding binary variables
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [15]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)
final_data.shape

(4261, 25)

In [16]:
final_data.columns

Index(['job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'certifications', 'salary',
       'employment_status_Independent or freelancer or company owner',
       'employment_status_Part time', 'job_title_Analytics consultant',
       'job_title_Architect', 'job_title_DBA', 'job_title_Data Scientist',
       'job_title_Developer', 'job_title_Engineer', 'job_title_Manager',
       'job_title_Other', 'job_title_Principal database engineer',
       'job_title_Sales', 'job_title_Sr Consultant ',
       'education_Bachelors (4 years)', 'education_Doctorate/PhD',
       'education_Masters', 'education_None (no degree completed)',
       'is_education_computer_related_Unknown',
       'is_education_computer_related_Yes'],
      dtype='object')

In [17]:
final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


## Train Test Split

In [19]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


## Pre-processing data

In [21]:
# selecting numerical features
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols


['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [22]:
# Applying standard scaling on numeric data
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [23]:
X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3896,-0.244386,0,2.385064,-0.017979,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3667,-1.250668,1,2.385064,-0.017979,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3031,0.258755,0,-0.482620,2.408654,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
3209,-0.244386,1,-0.482620,-0.624637,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
697,-0.244386,0,-0.482620,-0.624637,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0.258755,1,-0.482620,2.408654,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2414,2.774459,0,-0.482620,-0.624637,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2710,0.258755,0,-0.482620,-0.624637,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1665,0.258755,0,-0.482620,-0.624637,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0


In [24]:
#Fitting a Linear Regression Model
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [25]:
reg.coef_

array([ 1.75463988e+02,  3.76835523e+01,  2.00301945e+02,  2.16241961e+02,
        7.06905499e+01,  6.23180579e+01,  1.03884432e+02, -1.93267624e-12,
        1.24929638e+03,  9.62954083e+02,  8.45426864e+02,  6.70500904e+02,
        1.09857703e+03,  9.58470070e+02,  1.09065228e+02,  1.04758951e+03,
        8.16964243e+02,  3.66294929e+03,  3.51545676e+02, -2.35653812e+02,
        1.76338669e+02,  1.84222862e+02, -3.10240046e+02, -1.70130834e+02])

In [26]:
reg.intercept_

6052.645498017708

In [27]:
# Normalized MSE (Dividing by mean)
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

207.17314584313024

In [28]:
y_pred = reg.predict(X_test)

In [29]:
#Evaluating the model on test data
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Squared Error:", mse)


Mean Squared Error: 12337.647407044728


In [30]:
scaler.fit(X_test[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [31]:
X_test

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
1058,-0.115463,0,-0.510879,-0.632871,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1410,0.433278,0,-0.600090,0.576175,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
198,0.269360,1,-0.299886,-0.632871,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2851,0.270745,0,-0.510879,-0.028348,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
811,-0.227825,0,-0.510879,-0.632871,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808,-0.607300,1,0.632808,-0.632871,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1832,-1.723534,0,2.276968,-0.632871,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
152,-0.063055,0,0.031864,-0.632871,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2968,-1.224964,0,0.604260,-0.632871,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1


In [32]:
y_hat = reg.predict(X_test)

In [33]:
y_hat

array([6656.71642335, 7580.57245134, 6980.52373301, 7136.06021189,
       7294.30039227, 7475.07103461, 6873.61720085, 6673.48517514,
       7592.11990125, 6673.48517514, 7024.46595911, 6377.7671093 ,
       6800.36488794, 7259.3388177 , 5557.44617781, 6304.78858896,
       6391.14659889, 6901.06560582, 7225.09145545, 7031.78875445,
       6545.73626816, 6672.22941418, 7242.44547838, 7163.50861686,
       6358.25245314, 6282.03698043, 7138.34567113, 7179.05735679,
       7707.13360232, 7043.02061558, 6761.54999517, 7441.98813502,
       7180.98835221, 5971.655028  , 7044.81793285, 6844.6507783 ,
       7360.98806469, 6765.18754677, 6852.66313798, 7446.25707962,
       6854.03412775, 6579.54236013, 7057.63194505, 6608.23531735,
       7616.99940693, 7193.79341869, 7324.25419624, 6877.88990879,
       7864.95606163, 7005.33706325, 6873.37220079, 7764.16523604,
       7511.08418834, 6871.05543786, 8078.17238754, 7057.63194505,
       6405.45185698, 7585.91206599, 7075.36558068, 7057.63194

In [34]:
# Normalized Mean Absolute Error (Dividing by mean)
mae_test= mean_absolute_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Abosulte Error: ",mae_test)


Test Mean Abosulte Error:  0.12230887751760179


In [35]:
# Normalized Mean Squared Error (Dividing by mean)
mse_test = mean_squared_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Squared Error: ",mse_test)


Test Mean Squared Error:  216.766458891675
